In [ ]:
import time
import csv

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from datetime import date

driver = webdriver.Firefox()
namaRS = 'sari asih '
query = 'Rumah Sakit ' + namaRS + ' ciputat'
tgl = (date.today().strftime("%d/%m/%Y"))

driver.get('https://www.google.com/?hl=id')
driver.find_element_by_name('q').send_keys(query)
WebDriverWait(driver, 7).until(EC.element_to_be_clickable((By.NAME, 'btnK'))).click()

reviews_header = driver.find_element_by_css_selector('div.kp-header')
reviews_link = reviews_header.find_element_by_partial_link_text('ulasan Google')
number_of_reviews = int(reviews_link.text.split()[0])
reviews_link.click()

all_reviews = WebDriverWait(driver, 7).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[class*="WMbnJf gws-localreviews__google-review r-iXyMSdtUHaaI"]')))

allrevnum = len(all_reviews)
counter = 0

while len(all_reviews) < number_of_reviews:
    driver.execute_script('arguments[0].scrollIntoView(true);', all_reviews[-1])
    WebDriverWait(driver, 7, 0.25).until_not(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class$="activityIndicator"]')))
    all_reviews = driver.find_elements_by_css_selector('div[class*="gws-localreviews__general-reviews-block"]')
    
    if allrevnum == len(all_reviews):
        counter = counter+1
    else:
        allrevnum = len(all_reviews)
        counter = 1
    if counter == 5:
        break;
    
reviews = []
for review in all_reviews:
    item_review = []
    try:
        full_text_element = review.find_element_by_css_selector('span.review-full-text')
    except NoSuchElementException:
        full_text_element = review.find_element_by_css_selector('span[class^="r-"]')
    
    
    writer = review.find_element_by_css_selector('div[class="TSUbDb"]').get_attribute('textContent')
    date = review.find_element_by_css_selector('span[class="dehysf"]').get_attribute('textContent')
    star = review.find_element_by_css_selector('span[class^="fTKmHE99XE4__star"]').get_attribute('aria-label')
    star = star[13:14]
    
    item_review.append(writer)
    item_review.append(date)
    item_review.append(star)
    item_review.append(full_text_element.get_attribute('textContent'))
    item_review.append(tgl)
    
    reviews.append(item_review)
            
driver.quit()

print('Jumlah review: ' + str(number_of_reviews))    
print('Review terambil: ' + str(len(reviews)))
print('Data review: ')
print(reviews)



In [14]:
import csv
fileurl = 'review '+namaRS+'.csv'

with open(fileurl, "w",encoding='utf-8', newline ='') as f:
    writer = csv.writer(f)
    writer.writerows(reviews)
    
print('exported: '+fileurl)
print('-Selesai-')

exported: review Universitas Hasanuddin .csv
-Selesai-
